# ДЗ 8
Обучение сложных моделей машинного обучения с помощью распределенных инструментов

Цель: В данном проекте вы потренируетесь обучать модель градиентного бустинга и/или нейронной сети в распределенном режиме с помощью Spark


* 1) Подготовить признаки на основе датасета из домашнего задания модуля 2.
* 2) Обучить модель градиентного бустинга и/или нейронной сети на Spark.
* 3) Оценить качество модели.
### Критерии оценки: 
* 2 балла - Реализована подготовка признаков на основе датасета из домашнего задания модуля 2.
* 6 баллов - Обучена модель градиентного бустинга и/или нейронной сети на Spark.
* 2 балла - Проведена оценка качества модели.

# 1. Imports

In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.5`

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark.sql._
import org.apache.spark.sql.functions._

val spark = NotebookSparkSession
    .builder()
    .master("local[*]")
    .getOrCreate()

Loading spark-stubs
Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

import $ivy.$                                  


import org.apache.log4j.{Level, Logger}

import org.apache.spark.sql._

import org.apache.spark.sql.functions._


spark: SparkSession = org.apache.spark.sql.SparkSession@4defff80

In [2]:
import $ivy.`org.plotly-scala::plotly-almond:0.7.6`
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

repl.pprinter() = repl.pprinter().copy(defaultHeight = 3)

import $ivy.$                                      

import plotly._, plotly.element._, plotly.layout._, plotly.Almond._



In [3]:
def sc = spark.sparkContext
val sqlContext = new org.apache.spark.sql.SQLContext(sc)

defined function sc
sqlContext: SQLContext = org.apache.spark.sql.SQLContext@1d4fed30

In [4]:
import spark.implicits._
// val df = sqlContext.read.parquet("data/test/part-00000-6d949390-48b0-4104-a477-39e306b726c5-c000.gz.parquet")
// df.count()

import spark.implicits._
// val df = sqlContext.read.parquet("data/test/part-00000-6d949390-48b0-4104-a477-39e306b726c5-c000.gz.parquet")
// df.count()

In [5]:
import $ivy.`org.apache.spark::spark-mllib:2.4.5`

import org.apache.spark.ml.linalg.{Matrix, Vectors}
import org.apache.spark.ml.stat.Correlation
import org.apache.spark.ml.feature.{VectorAssembler, StandardScaler, MinMaxScaler}
import org.apache.spark.ml.classification.{GBTClassifier, MultilayerPerceptronClassifier}
import org.apache.spark.ml.tuning.{CrossValidator, ParamGridBuilder, TrainValidationSplit}
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator

import $ivy.$                                    


import org.apache.spark.ml.linalg.{Matrix, Vectors}

import org.apache.spark.ml.stat.Correlation

import org.apache.spark.ml.feature.{VectorAssembler, StandardScaler, MinMaxScaler}

import org.apache.spark.ml.classification.{GBTClassifier, MultilayerPerceptronClassifier}

import org.apache.spark.ml.tuning.{CrossValidator, ParamGridBuilder, TrainValidationSplit}

import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator

# 2. Read parquet

In [6]:
val df = sqlContext.read.parquet("collabTrain/")

parquet at cmd5.sc:1

48 / 48

parquet at cmd5.sc:1

1 / 1

df: DataFrame = [instanceId_userId: int, instanceId_objectType: string ... 167 more fields]

# 3. Some info about DataFrame

In [7]:
df.printSchema

root
 |-- instanceId_userId: integer (nullable = true)
 |-- instanceId_objectType: string (nullable = true)
 |-- instanceId_objectId: integer (nullable = true)
 |-- audit_pos: long (nullable = true)
 |-- audit_clientType: string (nullable = true)
 |-- audit_timestamp: long (nullable = true)
 |-- audit_timePassed: long (nullable = true)
 |-- audit_experiment: string (nullable = true)
 |-- audit_resourceType: long (nullable = true)
 |-- metadata_ownerId: integer (nullable = true)
 |-- metadata_ownerType: string (nullable = true)
 |-- metadata_createdAt: long (nullable = true)
 |-- metadata_authorId: integer (nullable = true)
 |-- metadata_applicationId: long (nullable = true)
 |-- metadata_numCompanions: integer (nullable = true)
 |-- metadata_numPhotos: integer (nullable = true)
 |-- metadata_numPolls: integer (nullable = true)
 |-- metadata_numSymbols: integer (nullable = true)
 |-- metadata_numTokens: integer (nullable = true)
 |-- metadata_numVideos: integer (nullable = true)
 |-- me

 |-- auditweights_userOwner_USER_INTERNAL_LIKE: double (nullable = true)
 |-- auditweights_userOwner_USER_INTERNAL_UNLIKE: double (nullable = true)
 |-- auditweights_userOwner_USER_PRESENT_SEND: double (nullable = true)
 |-- auditweights_userOwner_USER_PROFILE_VIEW: double (nullable = true)
 |-- auditweights_userOwner_USER_SEND_MESSAGE: double (nullable = true)
 |-- auditweights_userOwner_USER_STATUS_COMMENT_CREATE: double (nullable = true)
 |-- auditweights_userOwner_VIDEO: double (nullable = true)
 |-- auditweights_userOwner_VOTE_POLL: double (nullable = true)
 |-- auditweights_x_ActorsRelations: long (nullable = true)
 |-- auditweights_likersSvd_spark_hyper: double (nullable = true)
 |-- auditweights_source_PROMO: double (nullable = true)
 |-- date: date (nullable = true)



### 3.1. Rows count

In [8]:
val n_rows = df.count()

count at cmd7.sc:1

28 / 28

count at cmd7.sc:1

1 / 1

n_rows: Long = 18286575L

### 3.2. Columns count

In [9]:
val n_columns = df.columns.length

n_columns: Int = 169

# 4. Get target

In [10]:
val df_label_tmp = df
.withColumn("label", array_contains($"feedback", "Clicked").cast("Int"))
.withColumn("report_date", to_date(from_unixtime($"audit_timestamp" / 1000)))
.na.fill(0)

df_label_tmp: DataFrame = [instanceId_userId: int, instanceId_objectType: string ... 169 more fields]

In [11]:
df_label_tmp.select("report_date").distinct().orderBy(asc("report_date")).show()

show at cmd10.sc:1

28 / 28

show at cmd10.sc:1

200 / 200

+-----------+
|report_date|
+-----------+
| 2018-01-31|
| 2018-02-01|
| 2018-02-02|
| 2018-02-03|
| 2018-02-04|
| 2018-02-05|
| 2018-02-06|
| 2018-02-07|
| 2018-02-08|
| 2018-02-09|
| 2018-02-10|
| 2018-02-11|
| 2018-02-12|
| 2018-02-13|
| 2018-02-14|
| 2018-02-15|
| 2018-02-16|
| 2018-02-17|
| 2018-02-18|
| 2018-02-19|
+-----------+
only showing top 20 rows



### 4.1. Target info

In [12]:
df_label_tmp.select("label").describe().show()

describe at cmd11.sc:1

28 / 28

describe at cmd11.sc:1

1 / 1

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|           18286575|
|   mean|0.12351301432881773|
| stddev|0.32902515943356575|
|    min|                  0|
|    max|                  1|
+-------+-------------------+



### 4.2. Terget info about date

In [13]:
df_label_tmp.groupBy("report_date").agg(mean("label"), count("label")).show(50)

show at cmd12.sc:1

28 / 28

show at cmd12.sc:1

1 / 1

show at cmd12.sc:1

...

show at cmd12.sc:1

100 / 100

show at cmd12.sc:1

75 / 75

+-----------+-------------------+------------+
|report_date|         avg(label)|count(label)|
+-----------+-------------------+------------+
| 2018-03-17|0.12463131019359754|      388538|
| 2018-02-26|0.12457987721637222|      375783|
| 2018-02-22|0.12149496514329977|      387300|
| 2018-03-08|0.11261726835122482|      378299|
| 2018-02-11|0.12258548477839595|       15997|
| 2018-03-13|0.12108279478112426|      385447|
| 2018-02-04|0.12424823963285501|      412698|
| 2018-02-01|0.12544316880980694|      359908|
| 2018-02-23| 0.1187316868982838|      382240|
| 2018-02-27|0.12352490462803585|      370654|
| 2018-03-06|0.11870111156854672|      388640|
| 2018-02-02| 0.1263935336211776|      365707|
| 2018-02-12|0.12415237589864737|      391978|
| 2018-03-16|0.12774494212434828|      370363|
| 2018-03-19|0.12535450503860518|      393859|
| 2018-02-08|0.12695179552197702|      384902|
| 2018-02-10|0.12642492165332075|      364406|
| 2018-02-09|0.12812984754752083|      378413|
| 2018-03-04|

# 5. Get features for model

In [14]:
val feature_columns = scala.collection.mutable.ListBuffer[String]()
for (elem <- df.schema) {
    if ( (elem.name.toString.split("_")(0) == "userOwnerCounters") | (elem.name.toString.split("_")(0) == "ownerUserCounters")
       | (elem.name.toString.split("_")(0) == "auditweights") | (elem.name.toString.split("_")(0) == "user")
       | (elem.name.toString.split("_")(0) == "owner") 
    )
    feature_columns += elem.name.toString
}

feature_columns: collection.mutable.ListBuffer[String] = ListBuffer(
  "userOwnerCounters_USER_FEED_REMOVE",
...

In [15]:
feature_columns += "label"

res14: collection.mutable.ListBuffer[String] = ListBuffer(
  "userOwnerCounters_USER_FEED_REMOVE",
...

In [16]:
feature_columns.length

res15: Int = 139

# 6. Get vectors

In [17]:
val assembler = new VectorAssembler().setInputCols(feature_columns.filter(_ != "label").toArray).setOutputCol("features")
val output = assembler.transform(df_label_tmp).select("features", "label")

assembler: VectorAssembler = vecAssembler_3f34033efd43
output: DataFrame = [features: vector, label: int]

In [18]:
output.select("features", "label").show(5)

show at cmd17.sc:1

1 / 1

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(138,[22,23,24,25...|    0|
|(138,[18,23,24,25...|    1|
|(138,[67,68,69,70...|    0|
|(138,[67,68,69,70...|    0|
|(138,[67,68,69,70...|    0|
+--------------------+-----+
only showing top 5 rows



In [19]:
val scaler = new MinMaxScaler().setInputCol("features").setOutputCol("Scaled_features")
val output_scaled = scaler.fit(output).transform(output)
output_scaled.select("features","Scaled_features").show(5)

treeAggregate at RowMatrix.scala:433

28 / 28

treeAggregate at RowMatrix.scala:433

4 / 4

show at cmd18.sc:3

1 / 1

+--------------------+--------------------+
|            features|     Scaled_features|
+--------------------+--------------------+
|(138,[22,23,24,25...|[0.0,0.5,0.5,0.5,...|
|(138,[18,23,24,25...|[0.0,0.5,0.5,0.5,...|
|(138,[67,68,69,70...|[0.0,0.5,0.5,0.5,...|
|(138,[67,68,69,70...|[0.0,0.5,0.5,0.5,...|
|(138,[67,68,69,70...|[0.0,0.5,0.5,0.5,...|
+--------------------+--------------------+
only showing top 5 rows



scaler: MinMaxScaler = minMaxScal_91e9dbb455b9
output_scaled: DataFrame = [features: vector, label: int ... 1 more field]

In [20]:
val Array(training, test) = output_scaled.randomSplit(Array(0.6, 0.4), seed = 777)

training: Dataset[Row] = [features: vector, label: int ... 1 more field]
test: Dataset[Row] = [features: vector, label: int ... 1 more field]

# 7. Gradient boosting model

In [21]:
val gbt = new GBTClassifier()
.setFeaturesCol("features")
.setLabelCol("label")


gbt: GBTClassifier = gbtc_a49d1b8c2baf

### 7.1. CV

In [22]:
val paramGrid = new ParamGridBuilder()
  .addGrid(gbt.maxDepth, Array(3, 4, 5))
  .addGrid(gbt.maxIter, Array(50, 100, 250))
  .build()

paramGrid: Array[org.apache.spark.ml.param.ParamMap] = Array(
  {
...

In [23]:
// In this case the estimator is simply the linear regression.
// A TrainValidationSplit requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
val trainValidationSplit = new TrainValidationSplit()
  .setEstimator(gbt)
  .setEvaluator(new BinaryClassificationEvaluator)
  .setEstimatorParamMaps(paramGrid)
  // 50% of the data will be used for training and the remaining 50% for validation.
  .setTrainRatio(0.5)
  // Evaluate up to 4 parameter settings in parallel
  .setParallelism(4)

trainValidationSplit: TrainValidationSplit = tvs_0983dc1fa9b8

In [ ]:
val model_cv = trainValidationSplit.fit(training)

In [25]:
val predict_train_GB = model_cv.transform(training)
val predict_test_GB = model_cv.transform(test)

predict_train_GB: DataFrame = [features: vector, label: int ... 4 more fields]
predict_test_GB: DataFrame = [features: vector, label: int ... 4 more fields]

In [26]:
predict_test_GB.select("label", "rawPrediction", "probability", "prediction").show()

show at cmd25.sc:1

1 / 1

+-----+--------------------+--------------------+----------+
|label|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+----------+
|    0|[0.97942414663618...|[0.87640825729065...|       0.0|
|    0|[1.00431804999027...|[0.88170083536371...|       0.0|
|    0|[1.46518186674358...|[0.94932717536067...|       0.0|
|    0|[1.09558765336673...|[0.89945424674636...|       0.0|
|    0|[1.10732490672853...|[0.90155737678114...|       0.0|
|    0|[1.23996841494717...|[0.92272329363816...|       0.0|
|    0|[1.26257311664517...|[0.92588597012946...|       0.0|
|    0|[0.97244352161959...|[0.87488805486173...|       0.0|
|    0|[1.69849003427461...|[0.96761002156882...|       0.0|
|    0|[0.85089144960991...|[0.84576744837522...|       0.0|
|    0|[1.72352210728802...|[0.96914287044424...|       0.0|
|    0|[1.39739494994912...|[0.94239362990372...|       0.0|
|    0|[1.66792276275515...|[0.96563825899106...|       0.0|
|    1|[0.96799503806307

In [27]:
val evaluator = new BinaryClassificationEvaluator()
.setRawPredictionCol("rawPrediction")
.setLabelCol("label")

val lr_auc_train = evaluator.evaluate(predict_train_GB)
val lr_auc_test = evaluator.evaluate(predict_test_GB)

map at BinaryClassificationEvaluator.scala:81

28 / 28

sortByKey at BinaryClassificationMetrics.scala:155

28 / 28

combineByKey at BinaryClassificationMetrics.scala:154

...

collect at SlidingRDD.scala:78

30 / 30

aggregate at AreaUnderCurve.scala:45

29 / 29

evaluator: BinaryClassificationEvaluator = binEval_00e3900f355f
lr_auc_train: Double = 0.7545328698537831
lr_auc_test: Double = 0.7545061704811167

In [28]:
println("GB Roc auc train", lr_auc_train)

(GB Roc auc train,0.7545328698537831)


In [29]:
println("GB Roc auc test ", lr_auc_test)

(GB Roc auc test ,0.7545061704811167)


## Ниже иморт не работает (

In [18]:
import $ivy.`ml.dmlc::xgboost4j_2.09`

Downloaded https://repo1.maven.org/maven2/ml/dmlc//xgboost4j_2.09/-xgboost4j_2.09.pom
Downloaded https://repo1.maven.org/maven2/ml/dmlc//xgboost4j_2.09/-xgboost4j_2.09.pom.sha1
Failed to resolve ivy dependencies:Error downloading ml.dmlc::xgboost4j_2.09
  not found: /home/jovyan/.ivy2/local/ml.dmlc/xgboost4j_2.09/ivys/ivy.xml
  not found: https://repo1.maven.org/maven2/ml/dmlc//xgboost4j_2.09/-xgboost4j_2.09.pom

: 

In [ ]:
import $ivy.`org.apache.spark::spark-sql:2.4.5`